# asnn-goose v11: ALL FOUR ADVANCED TECHNIQUES (448d × 6L, ~37M params)

## abstract

v11 implements **ALL FOUR** advanced distillation techniques to achieve better PPL while maintaining faster training times than v10.

**v10 results (baseline for v11):**
- teacher ppl: 44.6
- student ppl: **526.4** (improved from v9's 541.7)
- tests: 9/9 passed
- training time: 26 min (too slow!)
- VRAM: 3.98GB

**v11: ALL FOUR ADVANCED TECHNIQUES**

| # | Technique | Paper | Key Benefit |
|---|-----------|-------|-------------|
| 1 | **Learnable Temperature (CTKD)** | ArXiv 2211.16231 | Adaptive difficulty |
| 2 | **Light Hidden Alignment** | TinyBERT | Better representations |
| 3 | **Progressive Training Stages** | POCL (ArXiv 2506.05695) | Curriculum learning |
| 4 | **Channel-wise Ternary Spikes** | TerViT/TTQ | Per-channel adaptivity |

**architecture (balanced for speed):**

| attribute | v10 | v11 | change |
|-----------|-----|-----|--------|
| d_model | 512 | **448** | -12% |
| n_layers | 8 | **6** | -25% |
| params | ~51M | **~37M** | -27% |
| training time | 26 min | **~12 min** | -54% |

**target:**
- student ppl: **<510** (balanced with training speed)
- tests: **12/12** passed (9 existing + 3 new)
- training time: ~12 minutes

---

**eptesicus laboratories - lumis-next initiative**

### references
- li et al. (2023) "curriculum temperature for knowledge distillation" (CTKD)
- jiao et al. (2020) "tinybert: distilling bert for natural language understanding"
- wu et al. (2025) "progressive overload curriculum learning" (POCL)
- ternarization: TTQ (2017), TerViT (2023)
- hinton et al. (2015) "distilling the knowledge in a neural network"
- radford et al. (2019) "language models are unsupervised multitask learners" (gpt-2)


In [ ]:
# =============================================================================
# cell 1: environment setup (v10 - added compile flag)
# =============================================================================
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import sys
import time
import math
import json
import base64
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass, asdict
from typing import Dict, List, Optional, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# v10: torch.compile flag (disable if causing issues)
USE_TORCH_COMPILE = True
USE_GRADIENT_CHECKPOINTING = True

# generate timestamp for this run
RUN_TIMESTAMP = datetime.now().strftime('%Y-%m-%d_%H%M%S')
print(f"run timestamp: {RUN_TIMESTAMP}")

# detect platform
IS_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
IS_COLAB = 'COLAB_GPU' in os.environ or 'google.colab' in sys.modules
PLATFORM = 'kaggle' if IS_KAGGLE else 'colab' if IS_COLAB else 'local'
OUTPUT_DIR = '/kaggle/working/outputs' if IS_KAGGLE else 'outputs'

for subdir in ['figures', 'checkpoints', 'logs', 'results']:
    os.makedirs(f'{OUTPUT_DIR}/{subdir}', exist_ok=True)

print(f"platform: {PLATFORM}")
print(f"output directory: {OUTPUT_DIR}")
print(f"torch.compile: {'enabled' if USE_TORCH_COMPILE else 'disabled'}")
print(f"gradient checkpointing: {'enabled' if USE_GRADIENT_CHECKPOINTING else 'disabled'}")

In [ ]:
# =============================================================================
# cell 2: pytorch and hardware setup (v10 - added compile precision)
# =============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.checkpoint import checkpoint
import numpy as np

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED = 42

torch.manual_seed(SEED)
np.random.seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cudnn.benchmark = True

    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"gpu: {gpu_name}")
    print(f"memory: {gpu_memory:.1f} gb")

# v10: set float32 matmul precision for torch.compile
if USE_TORCH_COMPILE and hasattr(torch, 'set_float32_matmul_precision'):
    torch.set_float32_matmul_precision('high')
    print("float32 matmul precision: high (for torch.compile)")

print(f"device: {DEVICE}")
print(f"pytorch: {torch.__version__}")

# check torch.compile availability
TORCH_COMPILE_AVAILABLE = hasattr(torch, 'compile') and torch.__version__ >= '2.0'
print(f"torch.compile available: {TORCH_COMPILE_AVAILABLE}")

## 1. v11 design: ALL FOUR ADVANCED TECHNIQUES

### 1.1 rationale for v11

v10 achieved PPL 526.4 with 100M parameters - a 3% improvement over v9. Research shows that raw capacity alone has diminishing returns. V11 shifts focus to **advanced distillation techniques** rather than model size:

- **d_model**: 512 → 448 (-12% to reduce overfitting risk)
- **n_layers**: 8 → 6 (-25% with better alignment)
- **params**: ~100M → ~37M (-63% but smarter training)

### 1.2 four advanced techniques

| technique | paper | implementation |
|-----------|-------|----------------|
| **Learnable Temperature (CTKD)** | ArXiv 2211.16231 | adaptive T during training |
| **Light Hidden Alignment** | TinyBERT 1909.10351 | weight=0.01 (100x lighter than v7) |
| **Progressive Stages (POCL)** | ArXiv 2506.05695 | 3-stage curriculum |
| **Channel-wise Ternary** | TerViT/TTQ | per-channel alpha and amplitude |

### 1.3 progressive training stages

| stage | steps | temperature | alignment mult | description |
|-------|-------|-------------|----------------|-------------|
| **1** | 0-40% | 4.0 (soft) | 0.0 | soft targets only |
| **2** | 40-70% | 2.5 (medium) | 0.5 | introduce light alignment |
| **3** | 70-100% | 1.5 (hard) | 1.0 | full training |

### 1.4 v11 changes from v10

| component | v10 | v11 | action |
|-----------|-----|-----|--------|
| d_model | 512 | **448** | **shrink** |
| n_layers | 8 | **6** | **shrink** |
| params | ~100M | **~37M** | **result** |
| temperature | fixed 2.0 | **learnable (CTKD)** | **upgrade** |
| hidden_align_weight | 0.0 | **0.01** | **enable** |
| progressive stages | none | **3 stages (POCL)** | **add** |
| ternary spikes | global | **channel-wise** | **upgrade** |
| teacher_indices | [1,2,4,5,7,8,10,11] | **[2,4,6,8,10,12]** | **remap for 6L** |
| gradient checkpointing | enabled | enabled | keep |
| torch.compile | enabled | enabled | keep |
| target PPL | <510 | **<510** | keep |

In [ ]:
# =============================================================================
# cell 4: configuration (v11 - ALL FOUR ADVANCED TECHNIQUES)
# =============================================================================
@dataclass
class Config:
    # gpt-2 teacher (frozen, pre-trained)
    teacher_name: str = "gpt2"

    # student model architecture - v11: BALANCED MODEL (~37M)
    d_model: int = 448      # v10: 512 -> v11: 448 (-12%)
    n_layers: int = 6       # v10: 8 -> v11: 6 (-25%)
    vocab_size: int = 50257
    max_seq_len: int = 256

    # distillation training
    distill_steps: int = 3000
    distill_lr: float = 3e-4
    temperature: float = 2.0      # initial temperature (learnable in v11)
    warmup_steps: int = 50        # minimal warmup

    # v11: gradient accumulation for effective larger batch
    accumulation_steps: int = 2   # effective batch = 8 * 2 = 16

    # v11: HIDDEN-STATE ALIGNMENT RE-ENABLED (100x lighter than v7)
    hidden_align_weight: float = 0.01  # v7 used 1.0 (failed), v8-v10 used 0.0
    teacher_d_model: int = 768         # gpt-2 hidden dim
    teacher_n_layers: int = 12         # gpt-2 layers

    # v11: ADVANCED TECHNIQUE FLAGS
    use_learnable_temperature: bool = True   # CTKD paper
    use_channel_wise_spikes: bool = True     # TerViT/TTQ inspired
    use_progressive_stages: bool = True      # POCL paper
    temperature_lr: float = 0.01             # separate LR for temperature

    # lora for ttt
    lora_rank: int = 8
    lora_alpha: float = 16.0
    ttt_lr: float = 1e-4
    ttt_steps: int = 100

    # spiking parameters
    spike_alpha: float = 1.0

    # general training
    batch_size: int = 8
    max_grad_norm: float = 1.0
    eval_interval: int = 100

config = Config()

print(f"configuration (v11 - ALL FOUR ADVANCED TECHNIQUES):")
print(f"  teacher: {config.teacher_name} (124m params)")
print(f"  student: d={config.d_model}, layers={config.n_layers}")
print(f"")
print(f"v11 ADVANCED TECHNIQUES:")
print(f"  1. learnable temperature (CTKD): {config.use_learnable_temperature}")
print(f"     - initial temp: {config.temperature}")
print(f"     - temp LR: {config.temperature_lr}")
print(f"  2. channel-wise spikes (TerViT): {config.use_channel_wise_spikes}")
print(f"  3. progressive stages (POCL): {config.use_progressive_stages}")
print(f"  4. hidden alignment: weight={config.hidden_align_weight} (v7 used 1.0, FAILED)")
print(f"")
print(f"training:")
print(f"  distillation: {config.distill_steps} steps")
print(f"  warmup: {config.warmup_steps} steps")
print(f"  accumulation: {config.accumulation_steps} (effective batch = {config.batch_size * config.accumulation_steps})")
print(f"")
print(f"speedups (same as v10):")
print(f"  gradient checkpointing: {USE_GRADIENT_CHECKPOINTING}")
print(f"  torch.compile: {USE_TORCH_COMPILE}")
print(f"  fused optimizer: enabled")
print(f"")
print(f"targets:")
print(f"  PPL: <510 (balanced with training speed)")
print(f"  training time: ~12 min")
print(f"  tests: 12/12 (9 existing + 3 new)")


## 2. trainable ternary spiking (v11: channel-wise)

### 2.1 motivation

fixed ternary values {-1, 0, +1} limit expressivity. the ternary spike paper (wei et al., 2023) shows that **trainable amplitude factors** per layer significantly improve accuracy.

**v11 extension**: channel-wise thresholds and amplitudes (TerViT/TTQ inspired)

### 2.2 implementation (v11 - channel-wise)

each layer has learnable **per-channel** parameters:
- `alpha[d_model]` - per-channel threshold scaling
- `amplitude[d_model]` - per-channel output scaling

```python
# channel-wise threshold (v11)
threshold = alpha * x.abs().mean(dim=(0, 1), keepdim=True)  # per-channel

# ternary spikes with per-channel amplitude
spikes = torch.zeros_like(x)
spikes[x > threshold] = +amplitude  # shape: [d_model]
spikes[x < -threshold] = -amplitude  # shape: [d_model]
```

### 2.3 straight-through estimator (ste)

$$\frac{\partial \mathcal{L}}{\partial x} = \frac{\partial \mathcal{L}}{\partial s}$$

gradient flows through the discrete spike operation using identity in backward pass.

### 2.4 channel amplitude regularization (v11)

to prevent channel amplitudes from diverging:

$$\mathcal{L}_{\text{channel\_reg}} = 0.01 \cdot \text{Var}(\text{amplitudes})$$

In [ ]:
# =============================================================================
# cell 6: v11 advanced techniques - CTKD + Channel-wise Ternary Spikes + Progressive Stages
# =============================================================================

# -----------------------------------------------------------------------------
# LearnableTemperature (CTKD - ArXiv 2211.16231)
# -----------------------------------------------------------------------------
class LearnableTemperature(nn.Module):
    """
    Learnable temperature for KL distillation (CTKD paper).
    
    Uses log-parameterization for smooth gradients.
    Clamped to [1.0, 10.0] for stability.
    
    Reference: "Curriculum Temperature for Knowledge Distillation" (ArXiv 2211.16231)
    """
    
    def __init__(self, init: float = 2.0):
        super().__init__()
        self.log_temp = nn.Parameter(torch.log(torch.tensor(init)))
    
    def forward(self) -> torch.Tensor:
        return torch.exp(self.log_temp).clamp(1.0, 10.0)
    
    def get_temperature(self) -> float:
        return self.forward().item()


# -----------------------------------------------------------------------------
# ChannelWiseTernarySpike (TerViT/TTQ inspired)
# -----------------------------------------------------------------------------
class ChannelWiseTernarySpike(nn.Module):
    """
    Per-channel learnable alpha and amplitude for ternary spikes.
    Each channel has its own threshold and output magnitude.
    
    Compared to TrainableTernarySpike (scalar amplitude), this version
    learns per-channel parameters for better expressivity.
    
    References:
    - TerViT: Channel-wise ternary ViT
    - TTQ: Trained Ternary Quantization (OpenReview 2017)
    """
    
    def __init__(self, d_model: int, alpha_init: float = 1.0):
        super().__init__()
        self.d_model = d_model
        # Per-channel parameters
        self.alpha = nn.Parameter(torch.ones(d_model) * alpha_init)
        self.amplitude = nn.Parameter(torch.ones(d_model))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: [batch, seq, d_model]
        # Per-channel threshold based on activation statistics
        x_abs_mean = x.abs().mean(dim=(0, 1), keepdim=True)  # [1, 1, d_model]
        threshold = self.alpha * x_abs_mean
        threshold = threshold.clamp(min=0.01, max=10.0)
        
        # Compute spike pattern without gradient (STE)
        with torch.no_grad():
            pos_mask = (x > threshold).float()
            neg_mask = (x < -threshold).float()
            spike_signs = pos_mask - neg_mask
        
        # Apply per-channel amplitude with STE gradient trick
        spikes = self.amplitude * spike_signs
        return spikes + (x - x.detach())
    
    def get_amplitude(self) -> float:
        """Return mean amplitude for compatibility with existing code."""
        return self.amplitude.mean().item()
    
    def get_stats(self) -> dict:
        """Return channel-wise statistics."""
        return {
            'alpha_mean': self.alpha.mean().item(),
            'alpha_std': self.alpha.std().item(),
            'amplitude_mean': self.amplitude.mean().item(),
            'amplitude_std': self.amplitude.std().item(),
            'amplitude_min': self.amplitude.min().item(),
            'amplitude_max': self.amplitude.max().item(),
        }


# -----------------------------------------------------------------------------
# TrainableTernarySpike (kept for backward compatibility / comparison)
# -----------------------------------------------------------------------------
class TrainableTernarySpike(nn.Module):
    """
    Original trainable ternary spike with scalar amplitude (from v8).
    Kept for backward compatibility and ablation studies.
    """

    def __init__(self, alpha: float = 1.0):
        super().__init__()
        self.alpha = alpha
        self.amplitude = nn.Parameter(torch.ones(1))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        threshold = self.alpha * x.abs().mean(dim=-1, keepdim=True)
        threshold = threshold.clamp(min=0.01, max=10.0)

        with torch.no_grad():
            pos_mask = (x > threshold).float()
            neg_mask = (x < -threshold).float()
            spike_signs = pos_mask - neg_mask

        spikes = self.amplitude * spike_signs
        return spikes + (x - x.detach())

    def get_amplitude(self) -> float:
        return self.amplitude.item()


# -----------------------------------------------------------------------------
# Progressive Training Stages (POCL - ArXiv 2506.05695)
# -----------------------------------------------------------------------------
def get_stage_params(step: int, total_steps: int = 3000) -> dict:
    """
    3-stage progressive training schedule for curriculum learning.

    Stage 1 (0-40%): Soft targets, no alignment (easier)
    Stage 2 (40-70%): Medium difficulty
    Stage 3 (70-100%): Hard targets, full alignment

    Reference: "Progressive Overload Curriculum Learning" (ArXiv 2506.05695)

    Args:
        step: Current training step
        total_steps: Total number of training steps

    Returns:
        Dictionary with stage parameters:
        - stage: Stage number (1, 2, or 3)
        - temp_target: Target temperature for this stage
        - align_mult: Multiplier for alignment weight (0.0, 0.5, or 1.0)
    """
    if step < total_steps * 0.4:  # Stage 1: 0-40%
        return {
            'stage': 1,
            'temp_target': 4.0,      # Soft targets initially
            'align_mult': 0.0,       # No alignment
        }
    elif step < total_steps * 0.7:  # Stage 2: 40-70%
        return {
            'stage': 2,
            'temp_target': 2.5,      # Medium difficulty
            'align_mult': 0.5,       # Half alignment
        }
    else:  # Stage 3: 70-100%
        return {
            'stage': 3,
            'temp_target': 1.5,      # Hard targets
            'align_mult': 1.0,       # Full alignment
        }


# -----------------------------------------------------------------------------
# Test new classes
# -----------------------------------------------------------------------------
print("testing v11 advanced techniques...")

# Test LearnableTemperature
print("\n1. LearnableTemperature (CTKD):")
_temp = LearnableTemperature(init=2.0).to(DEVICE)
print(f"   initial temperature: {_temp.get_temperature():.4f}")
# Verify gradient flows
_t = _temp()
_t.backward()
print(f"   gradient exists: {_temp.log_temp.grad is not None}")
del _temp, _t

# Test ChannelWiseTernarySpike
print("\n2. ChannelWiseTernarySpike:")
_spike = ChannelWiseTernarySpike(d_model=64).to(DEVICE)
_x = torch.randn(2, 16, 64, device=DEVICE, requires_grad=True)
_y = _spike(_x)
_y.sum().backward()
stats = _spike.get_stats()
print(f"   amplitude mean: {stats['amplitude_mean']:.4f}")
print(f"   gradient for x: {'exists' if _x.grad is not None else 'none'}")
print(f"   gradient for amplitude: {_spike.amplitude.grad is not None}")
del _spike, _x, _y

# Test progressive stages
print("\n3. Progressive Training Stages (POCL):")
for test_step in [0, 1200, 2100, 2999]:
    params = get_stage_params(test_step, 3000)
    print(f"   step {test_step}: stage {params['stage']}, temp={params['temp_target']}, align={params['align_mult']}")

print("\nv11 advanced techniques ready!")


In [ ]:
# =============================================================================
# cell 7: hardware and spike stats collectors (same as v9)
# =============================================================================
class HardwareStatsCollector:
    """collect gpu memory, timing, and throughput metrics."""

    def __init__(self):
        self.gpu_memory_history = []
        self.step_times = []
        self.tokens_processed = 0
        self.start_time = None

    def start(self):
        self.start_time = time.time()
        if torch.cuda.is_available():
            torch.cuda.reset_peak_memory_stats()

    def record_step(self, batch_size: int, seq_len: int):
        if torch.cuda.is_available():
            self.gpu_memory_history.append(torch.cuda.memory_allocated() / 1e9)
        self.tokens_processed += batch_size * seq_len
        self.step_times.append(time.time())

    def get_throughput(self) -> float:
        if len(self.step_times) < 2:
            return 0.0
        elapsed = self.step_times[-1] - self.step_times[0]
        return self.tokens_processed / elapsed if elapsed > 0 else 0.0

    def get_summary(self) -> Dict[str, Any]:
        elapsed = time.time() - self.start_time if self.start_time else 0
        return {
            'peak_gpu_memory_gb': max(self.gpu_memory_history) if self.gpu_memory_history else 0,
            'avg_gpu_memory_gb': float(np.mean(self.gpu_memory_history)) if self.gpu_memory_history else 0,
            'total_training_time_s': elapsed,
            'total_training_time_min': elapsed / 60,
            'tokens_processed': self.tokens_processed,
            'throughput_tokens_per_sec': self.get_throughput(),
        }


class SpikeStatsCollector:
    """collect per-layer spike density and amplitude evolution."""

    def __init__(self, n_layers: int):
        self.n_layers = n_layers
        self.density_history = {i: {'k': [], 'v': []} for i in range(n_layers)}
        self.amplitude_history = {i: {'k': [], 'v': []} for i in range(n_layers)}
        self.step_densities = []

    def record(self, student, step: int):
        stats = student.get_spike_stats()
        all_densities = []
        for i in range(self.n_layers):
            layer_key = f'layer_{i}'
            if layer_key in stats:
                k_density = stats[layer_key].get('k', 0)
                v_density = stats[layer_key].get('v', 0)
                k_amp = stats[layer_key].get('k_amp', 1.0)
                v_amp = stats[layer_key].get('v_amp', 1.0)

                self.density_history[i]['k'].append(k_density)
                self.density_history[i]['v'].append(v_density)
                self.amplitude_history[i]['k'].append(k_amp)
                self.amplitude_history[i]['v'].append(v_amp)
                all_densities.extend([k_density, v_density])

        if all_densities:
            self.step_densities.append({'step': step, 'density': float(np.mean(all_densities))})

    def get_summary(self) -> Dict[str, Any]:
        per_layer = {}
        all_k, all_v = [], []
        all_k_amp, all_v_amp = [], []

        for i in range(self.n_layers):
            k_vals = self.density_history[i]['k']
            v_vals = self.density_history[i]['v']
            k_amps = self.amplitude_history[i]['k']
            v_amps = self.amplitude_history[i]['v']

            per_layer[f'layer_{i}'] = {
                'k_mean': float(np.mean(k_vals)) if k_vals else 0,
                'k_std': float(np.std(k_vals)) if k_vals else 0,
                'k_final': float(k_vals[-1]) if k_vals else 0,
                'v_mean': float(np.mean(v_vals)) if v_vals else 0,
                'v_std': float(np.std(v_vals)) if v_vals else 0,
                'v_final': float(v_vals[-1]) if v_vals else 0,
                'k_amp_final': float(k_amps[-1]) if k_amps else 1.0,
                'v_amp_final': float(v_amps[-1]) if v_amps else 1.0,
            }
            all_k.extend(k_vals)
            all_v.extend(v_vals)
            if k_amps: all_k_amp.append(k_amps[-1])
            if v_amps: all_v_amp.append(v_amps[-1])

        return {
            'per_layer': per_layer,
            'overall_k_density': float(np.mean(all_k)) if all_k else 0,
            'overall_v_density': float(np.mean(all_v)) if all_v else 0,
            'overall_density': float(np.mean(all_k + all_v)) if (all_k or all_v) else 0,
            'amplitudes': {'k': all_k_amp, 'v': all_v_amp},
            'density_history': self.step_densities,
        }

print("collectors defined")

In [ ]:
# =============================================================================
# cell 8: spiking goose model (v11 - channel-wise spikes + gradient checkpointing)
# =============================================================================
class SpikingGooseRecurrentLayer(nn.Module):
    """
    RWKV-style recurrence with trainable ternary spiking.
    
    v11: Supports channel-wise ternary spikes (when use_channel_wise=True)
    """

    def __init__(self, d_model, layer_idx=0, n_layers=4, spike_alpha=1.0, 
                 use_channel_wise: bool = False):
        super().__init__()
        self.d_model = d_model
        self.layer_idx = layer_idx
        self.use_channel_wise = use_channel_wise
        self.ln = nn.LayerNorm(d_model)

        ratio = layer_idx / max(n_layers - 1, 1)
        self.time_mix_k = nn.Parameter(torch.ones(d_model) * (1 - ratio))
        self.time_mix_v = nn.Parameter(torch.ones(d_model) * (1 - ratio))
        self.time_mix_r = nn.Parameter(torch.ones(d_model) * (1 - ratio))
        self.decay_weight = nn.Parameter(torch.zeros(d_model) - 0.5)

        self.key_proj = nn.Linear(d_model, d_model, bias=False)
        self.value_proj = nn.Linear(d_model, d_model, bias=False)
        self.receptance_proj = nn.Linear(d_model, d_model, bias=False)
        self.output_proj = nn.Linear(d_model, d_model, bias=False)

        # v11: Use channel-wise spikes if enabled
        if use_channel_wise:
            self.k_spike = ChannelWiseTernarySpike(d_model, alpha_init=spike_alpha)
            self.v_spike = ChannelWiseTernarySpike(d_model, alpha_init=spike_alpha)
        else:
            self.k_spike = TrainableTernarySpike(alpha=spike_alpha)
            self.v_spike = TrainableTernarySpike(alpha=spike_alpha)

        self.register_buffer('running_k_density', torch.tensor(0.0))
        self.register_buffer('running_v_density', torch.tensor(0.0))
        self._init_weights()

    def _init_weights(self):
        std = 0.1 / math.sqrt(self.d_model)
        for m in [self.key_proj, self.value_proj, self.receptance_proj, self.output_proj]:
            nn.init.normal_(m.weight, std=std)

    def forward(self, x):
        B, T, D = x.shape
        x_norm = self.ln(x)
        prev_x = F.pad(x_norm[:, :-1, :], (0, 0, 1, 0))

        xk = x_norm * self.time_mix_k + prev_x * (1 - self.time_mix_k)
        xv = x_norm * self.time_mix_v + prev_x * (1 - self.time_mix_v)
        xr = x_norm * self.time_mix_r + prev_x * (1 - self.time_mix_r)

        k_pre = self.key_proj(xk)
        v_pre = self.value_proj(xv)

        k = self.k_spike(k_pre)
        v = self.v_spike(v_pre)
        r = torch.sigmoid(self.receptance_proj(xr))

        kv = k * v
        decay = torch.sigmoid(self.decay_weight)
        t_idx = torch.arange(T, device=x.device, dtype=x.dtype)
        decay_powers = decay.unsqueeze(0) ** t_idx.unsqueeze(1)

        kv_weighted = kv / (decay_powers.unsqueeze(0) + 1e-8)
        S = torch.cumsum(kv_weighted, dim=1) * decay_powers.unsqueeze(0)

        if self.training:
            with torch.no_grad():
                self.running_k_density = 0.99 * self.running_k_density + 0.01 * (k != 0).float().mean()
                self.running_v_density = 0.99 * self.running_v_density + 0.01 * (v != 0).float().mean()

        return x + r * self.output_proj(S)

    def get_spike_density(self):
        return {
            'k': self.running_k_density.item(),
            'v': self.running_v_density.item(),
            'k_amp': self.k_spike.get_amplitude(),
            'v_amp': self.v_spike.get_amplitude(),
        }
    
    def get_channel_wise_stats(self) -> dict:
        """Get channel-wise spike statistics (only available if use_channel_wise=True)."""
        if self.use_channel_wise:
            return {
                'k': self.k_spike.get_stats(),
                'v': self.v_spike.get_stats(),
            }
        return None


class GooseFFN(nn.Module):
    def __init__(self, d_model, expand=4):
        super().__init__()
        self.ln = nn.LayerNorm(d_model)
        self.w1 = nn.Linear(d_model, d_model * expand, bias=False)
        self.w2 = nn.Linear(d_model * expand, d_model, bias=False)

    def forward(self, x):
        return x + self.w2(F.silu(self.w1(self.ln(x))))


class StudentSpikingGoose(nn.Module):
    """
    Spiking student model with trainable ternary activations.
    
    v11: Supports channel-wise ternary spikes + gradient checkpointing.
    """

    def __init__(self, cfg, use_checkpointing=True):
        super().__init__()
        self.cfg = cfg
        self.use_checkpointing = use_checkpointing and USE_GRADIENT_CHECKPOINTING
        
        # v11: Check for channel-wise spikes flag
        use_channel_wise = getattr(cfg, 'use_channel_wise_spikes', False)
        
        self.embed = nn.Embedding(cfg.vocab_size, cfg.d_model)
        self.pos_embed = nn.Embedding(cfg.max_seq_len, cfg.d_model)

        self.layers = nn.ModuleList([
            nn.ModuleDict({
                'rec': SpikingGooseRecurrentLayer(
                    cfg.d_model, i, cfg.n_layers, cfg.spike_alpha,
                    use_channel_wise=use_channel_wise
                ),
                'ffn': GooseFFN(cfg.d_model),
            })
            for i in range(cfg.n_layers)
        ])

        self.ln_out = nn.LayerNorm(cfg.d_model)
        self.head = nn.Linear(cfg.d_model, cfg.vocab_size, bias=False)
        self.head.weight = self.embed.weight

        nn.init.normal_(self.embed.weight, std=0.02)
        nn.init.normal_(self.pos_embed.weight, std=0.02)

    def _layer_forward(self, layer, x):
        """helper for gradient checkpointing - processes one layer."""
        x = layer['rec'](x)
        x = layer['ffn'](x)
        return x

    def forward(self, input_ids, return_hiddens=False):
        """forward pass with optional hidden state return for alignment."""
        B, T = input_ids.shape
        pos = torch.arange(T, device=input_ids.device).unsqueeze(0)
        x = self.embed(input_ids) + self.pos_embed(pos)

        hiddens = [x] if return_hiddens else None

        for layer in self.layers:
            if self.use_checkpointing and self.training:
                # v10: gradient checkpointing with use_reentrant=False (recommended)
                x = checkpoint(self._layer_forward, layer, x, use_reentrant=False)
            else:
                x = self._layer_forward(layer, x)
            
            if return_hiddens:
                hiddens.append(x)

        logits = self.head(self.ln_out(x))

        if return_hiddens:
            return logits, hiddens
        return logits

    def get_spike_stats(self):
        return {f'layer_{i}': layer['rec'].get_spike_density() for i, layer in enumerate(self.layers)}

    def get_avg_spike_density(self):
        densities = []
        for layer in self.layers:
            d = layer['rec'].get_spike_density()
            densities.extend([d['k'], d['v']])
        return float(np.mean(densities)) if densities else 0.0

    def get_amplitudes(self):
        return {f'layer_{i}': {'k': layer['rec'].k_spike.get_amplitude(), 'v': layer['rec'].v_spike.get_amplitude()}
                for i, layer in enumerate(self.layers)}
    
    def get_channel_amplitude_variance(self) -> float:
        """Get total variance of channel-wise amplitudes (for regularization)."""
        total_var = 0.0
        for layer in self.layers:
            rec = layer['rec']
            if hasattr(rec.k_spike, 'amplitude') and rec.k_spike.amplitude.numel() > 1:
                total_var += rec.k_spike.amplitude.var().item()
                total_var += rec.v_spike.amplitude.var().item()
        return total_var

print("student model defined (v11: channel-wise spikes + gradient checkpointing)")
print(f"  gradient checkpointing: {USE_GRADIENT_CHECKPOINTING}")
print(f"  channel-wise spikes: {config.use_channel_wise_spikes}")


In [ ]:
# =============================================================================
# cell 9: hidden-state alignment (v11 - updated for 6 layers)
# =============================================================================
class HiddenStateProjector(nn.Module):
    """
    Project student hidden states to teacher dimension for alignment.
    
    student: (B, T, 448) -> (B, T, 768)  # v11: 448 dim
    
    Maps 6 student layers to selected teacher layers.
    
    v11: Re-enabled with light weight (0.01) after v7's failure (weight=1.0).
    """

    def __init__(self, student_dim: int, teacher_dim: int, n_student_layers: int):
        super().__init__()
        self.projectors = nn.ModuleList([
            nn.Linear(student_dim, teacher_dim, bias=False)
            for _ in range(n_student_layers)
        ])
        for proj in self.projectors:
            nn.init.normal_(proj.weight, std=0.02)

    def forward(self, student_hidden: torch.Tensor, layer_idx: int) -> torch.Tensor:
        return self.projectors[layer_idx](student_hidden)


def compute_hidden_alignment_loss(
    teacher_hiddens: List[torch.Tensor],
    student_hiddens: List[torch.Tensor],
    projector: HiddenStateProjector,
    teacher_layers: int = 12,
    student_layers: int = 6,  # v11: 6 student layers
) -> torch.Tensor:
    """
    Compute MSE loss between projected student hiddens and teacher hiddens.
    
    v11 maps 6 student layers to 12 teacher layers (every 2nd teacher layer):
      student 0 -> teacher 2
      student 1 -> teacher 4
      student 2 -> teacher 6
      student 3 -> teacher 8
      student 4 -> teacher 10
      student 5 -> teacher 12
    
    v11: Re-enabled with weight=0.01 (100x lighter than v7's 1.0 which FAILED).
    """
    loss = 0.0
    # v11: updated mapping for 6 student layers (every 2nd teacher layer)
    teacher_indices = [2, 4, 6, 8, 10, 12]  # v10 was [1, 2, 4, 5, 7, 8, 10, 11]

    for s_idx, t_idx in enumerate(teacher_indices):
        if s_idx < len(student_hiddens) - 1 and t_idx < len(teacher_hiddens):
            s_h = student_hiddens[s_idx + 1]
            t_h = teacher_hiddens[t_idx]
            s_h_proj = projector(s_h, s_idx)
            loss += F.mse_loss(s_h_proj, t_h)

    return loss / len(teacher_indices)


print("hidden-state alignment defined (v11: 6 student layers)")
print(f"  student layers: {config.n_layers} (d={config.d_model})")
print(f"  teacher layers: {config.teacher_n_layers} (d={config.teacher_d_model})")
print(f"  layer mapping: [2, 4, 6, 8, 10, 12] (every 2nd teacher layer)")
print(f"  current weight: {config.hidden_align_weight} (v7 used 1.0 and FAILED)")


In [ ]:
# =============================================================================
# cell 10: cosine lr with warmup (same as v9)
# =============================================================================
def get_cosine_schedule_with_warmup(
    optimizer: torch.optim.Optimizer,
    warmup_steps: int,
    total_steps: int,
) -> torch.optim.lr_scheduler.LambdaLR:
    """
    linear warmup then cosine decay to 0.
    """
    def lr_lambda(step: int) -> float:
        if step < warmup_steps:
            return step / max(warmup_steps, 1)
        else:
            progress = (step - warmup_steps) / max(total_steps - warmup_steps, 1)
            return 0.5 * (1.0 + math.cos(math.pi * progress))

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)


print(f"cosine lr: {config.warmup_steps} warmup, {config.distill_steps} total")

In [ ]:
# =============================================================================
# cell 11: load gpt-2 teacher (same as v9)
# =============================================================================
from transformers import GPT2LMHeadModel, GPT2Tokenizer

print("loading gpt-2 teacher...")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

teacher = GPT2LMHeadModel.from_pretrained('gpt2').to(DEVICE)
teacher.eval()
for p in teacher.parameters():
    p.requires_grad = False

teacher_params = sum(p.numel() for p in teacher.parameters())
print(f"teacher: gpt-2 ({teacher_params:,} params)")

## 3. gpt-2 as teacher

### 3.1 model specifications

| attribute | gpt-2 (teacher) | asnn-goose v11 (student) |
|-----------|-----------------|-------------------------|
| parameters | 124m | **~37m** |
| layers | 12 | **6** |
| hidden dim | 768 | **448** |
| attention | softmax (dense) | linear + ternary spikes |
| ppl (wikitext-2) | ~30 | target: **<510** (v11) |

### 3.2 distillation loss

898\mathcal{L}_{	ext{total}} = \mathcal{L}_{	ext{kd}} + \lambda \cdot \mathcal{L}_{	ext{align}} + \mathcal{L}_{	ext{temp\_reg}} + \mathcal{L}_{	ext{channel\_reg}}898

where:
- $\mathcal{L}_{	ext{kd}} = T^2 \cdot 	ext{KL}(p^{(t)} \| p^{(s)})$ with learnable $ (CTKD)
- $\lambda = 0.01$ (light hidden alignment)
- temperature regularization pulls $ towards stage-specific targets

In [ ]:
# =============================================================================
# cell 13: data loading (v10 - efficient DataLoader)
# =============================================================================
from datasets import load_dataset

print("loading wikitext-2...")
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')

def pre_tokenize(texts, max_len):
    all_tokens = []
    for text in tqdm(texts, desc="tokenizing", leave=False):
        if text.strip():
            all_tokens.extend(tokenizer.encode(text, max_length=max_len*2, truncation=True))
    chunks = [all_tokens[i:i+max_len] for i in range(0, len(all_tokens)-max_len+1, max_len//2) if len(all_tokens[i:i+max_len]) == max_len]
    print(f"created {len(chunks)} sequences")
    return torch.tensor(chunks, dtype=torch.long)

train_tokens = pre_tokenize(dataset['train']['text'], config.max_seq_len)
val_tokens = pre_tokenize(dataset['validation']['text'], config.max_seq_len)

# v10: efficient DataLoader with workers and prefetch
# Note: num_workers=0 for Kaggle/Colab compatibility, but prefetch still helps
dataloader_kwargs = {
    'batch_size': config.batch_size,
    'pin_memory': True,
    'num_workers': 0 if IS_KAGGLE or IS_COLAB else 2,  # workers disabled on cloud platforms
    'prefetch_factor': None if IS_KAGGLE or IS_COLAB else 2,
    'persistent_workers': False if IS_KAGGLE or IS_COLAB else True,
}

train_loader = DataLoader(TensorDataset(train_tokens), shuffle=True, **dataloader_kwargs)
val_loader = DataLoader(TensorDataset(val_tokens), shuffle=False, **dataloader_kwargs)

print(f"train: {len(train_loader)} batches, val: {len(val_loader)} batches")
print(f"DataLoader: num_workers={dataloader_kwargs['num_workers']}, pin_memory={dataloader_kwargs['pin_memory']}")

In [ ]:
# =============================================================================
# cell 14: create student model and projector (v10 - with compile)
# =============================================================================
print("creating student model (v10 - 100M with speedups)...")

student = StudentSpikingGoose(config, use_checkpointing=USE_GRADIENT_CHECKPOINTING).to(DEVICE)
student_params = sum(p.numel() for p in student.parameters())

# v10: create projector (even if not used, for infrastructure preservation)
projector = HiddenStateProjector(
    student_dim=config.d_model,
    teacher_dim=config.teacher_d_model,
    n_student_layers=config.n_layers
).to(DEVICE)
projector_params = sum(p.numel() for p in projector.parameters())

compression_ratio = teacher_params / student_params

print(f"student: asnn-goose v10 ({student_params:,} params)")
print(f"projector: ({projector_params:,} params)")
print(f"compression ratio: {compression_ratio:.1f}x")
print(f"")
print(f"v10 architecture:")
print(f"  d_model: {config.d_model}")
print(f"  n_layers: {config.n_layers}")
print(f"  params: ~{student_params // 1_000_000}M")
print(f"")

# v10: compile model if available and enabled
compile_success = False
if USE_TORCH_COMPILE and TORCH_COMPILE_AVAILABLE:
    try:
        print("compiling student model with torch.compile...")
        # Use the compile() method as recommended by PyTorch docs
        student = torch.compile(student, mode='reduce-overhead')
        compile_success = True
        print("compilation successful!")
    except Exception as e:
        print(f"torch.compile failed: {e}")
        print("continuing without compilation")
else:
    print(f"torch.compile skipped (USE_TORCH_COMPILE={USE_TORCH_COMPILE}, available={TORCH_COMPILE_AVAILABLE})")

print(f"")
print(f"speedups active:")
print(f"  gradient checkpointing: {USE_GRADIENT_CHECKPOINTING}")
print(f"  torch.compile: {compile_success}")
print(f"  accumulation_steps: {config.accumulation_steps}")

In [ ]:
# =============================================================================
# cell 15: evaluation functions (same as v9)
# =============================================================================
@torch.no_grad()
def evaluate(model, loader, device, is_gpt2=False):
    model.eval()
    total_loss, total_tokens = 0, 0
    for batch in loader:
        ids = batch[0].to(device)
        with torch.cuda.amp.autocast():
            logits = model(ids).logits if is_gpt2 else model(ids)
        loss = F.cross_entropy(logits[:, :-1].reshape(-1, logits.size(-1)), ids[:, 1:].reshape(-1), reduction='sum')
        total_loss += loss.item()
        total_tokens += ids[:, 1:].numel()
    return total_loss / total_tokens

def get_ppl(loss):
    return math.exp(min(loss, 10))

print("evaluation functions defined")

## 4. distillation training (v11)

### 4.1 loss function (v11 - four components)

103\mathcal{L}_{\text{total}} = \mathcal{L}_{\text{kd}} + \lambda \cdot \mathcal{L}_{\text{align}} + \mathcal{L}_{\text{temp\_reg}} + \mathcal{L}_{\text{channel\_reg}}103

where:
- $\mathcal{L}_{\text{kd}} = T^2 \cdot \text{KL}(p^{(t)} \| p^{(s)})$ with **learnable T (CTKD)**
- $\lambda = 0.01$ (light hidden alignment, 100x lighter than v7)
- $\mathcal{L}_{\text{temp\_reg}} = 0.1 \cdot (T - T_{target})^2$ (soft constraint to stage target)
- $\mathcal{L}_{\text{channel\_reg}} = 0.01 \cdot \text{Var}(\text{amplitudes})$ (prevent divergence)

### 4.2 four advanced techniques

| technique | paper | implementation |
|-----------|-------|----------------|
| **Learnable Temperature** | CTKD (ArXiv 2211.16231) | log-parameterized T, clamped [1,10] |
| **Light Hidden Alignment** | TinyBERT (ArXiv 1909.10351) | projector 448→768, weight=0.01 |
| **Progressive Stages** | POCL (ArXiv 2506.05695) | 3 stages: 40%/30%/30% |
| **Channel-wise Ternary** | TerViT/TTQ | per-channel alpha & amplitude |

### 4.3 v11 changes from v10

| aspect | v10 | v11 |
|--------|-----|-----|
| d_model | 512 | **448** |
| n_layers | 8 | **6** |
| params | ~100M | **~37M** |
| temperature | fixed 2.0 | **learnable (CTKD)** |
| hidden_align_weight | 0.0 | **0.01** |
| progressive stages | none | **3 stages (POCL)** |
| ternary spikes | global | **channel-wise** |
| target PPL | <510 | **<510** |

In [ ]:
# =============================================================================
# cell 17: distillation training loop (v11 - ALL FOUR ADVANCED TECHNIQUES)
# =============================================================================
def distill_v11(teacher, student, projector, train_loader, val_loader, cfg, device,
               hw_stats, spike_stats):
    """
    v11 distillation with ALL FOUR advanced techniques:
    
    1. Learnable Temperature (CTKD) - ArXiv 2211.16231
    2. Light Hidden Alignment - weight=0.01 (100x lighter than v7)
    3. Progressive Training Stages (POCL) - ArXiv 2506.05695
    4. Channel-wise Ternary Spikes - handled by model, add regularization
    
    Key improvements over v10:
    - Temperature adapts during training
    - Alignment weight modulated by stage
    - Per-channel amplitude regularization
    """
    training_logs = {
        'loss_history': [],
        'kl_loss_history': [],
        'align_loss_history': [],
        'ppl_history': [],
        'lr_history': [],
        # v11: new tracking
        'temp_history': [],
        'stage_history': [],
        'channel_reg_history': [],
    }

    # =========================================================================
    # v11 TECHNIQUE 1: Learnable Temperature (CTKD)
    # =========================================================================
    if cfg.use_learnable_temperature:
        temp_module = LearnableTemperature(init=cfg.temperature).to(device)
        print(f"using learnable temperature (CTKD): init={cfg.temperature}")
    else:
        temp_module = None
        print(f"using fixed temperature: {cfg.temperature}")

    # =========================================================================
    # Setup optimizer with separate param groups
    # =========================================================================
    param_groups = [
        {'params': list(student.parameters()), 'lr': cfg.distill_lr}
    ]
    
    # Add projector params if alignment is enabled
    if cfg.hidden_align_weight > 0:
        param_groups.append({'params': list(projector.parameters()), 'lr': cfg.distill_lr})
        print(f"hidden alignment enabled: weight={cfg.hidden_align_weight}")
    
    # Add temperature params with separate LR
    if temp_module is not None:
        param_groups.append({'params': list(temp_module.parameters()), 'lr': cfg.temperature_lr})
        print(f"temperature LR: {cfg.temperature_lr}")
    
    # Collect all params for gradient clipping
    all_params = []
    for group in param_groups:
        all_params.extend(group['params'])
    
    # Fused AdamW for speedup (PyTorch 2.0+)
    try:
        optimizer = torch.optim.AdamW(param_groups, weight_decay=0.01, fused=True)
        print("using fused AdamW")
    except TypeError:
        optimizer = torch.optim.AdamW(param_groups, weight_decay=0.01)
        print("fused AdamW not available, using standard")
    
    scheduler = get_cosine_schedule_with_warmup(optimizer, cfg.warmup_steps, cfg.distill_steps)
    scaler = torch.cuda.amp.GradScaler()

    hw_stats.start()
    step = 0
    accum_step = 0
    best_val = float('inf')
    current_stage = 1

    # Gradient accumulation
    accumulation_steps = cfg.accumulation_steps
    effective_batch = cfg.batch_size * accumulation_steps
    print(f"gradient accumulation: {accumulation_steps} steps (effective batch = {effective_batch})")
    
    # v11: Progressive stages info
    if cfg.use_progressive_stages:
        print("progressive training stages (POCL) enabled:")
        print("  stage 1 (0-40%): temp_target=4.0, align=0%")
        print("  stage 2 (40-70%): temp_target=2.5, align=50%")
        print("  stage 3 (70-100%): temp_target=1.5, align=100%")

    pbar = tqdm(total=cfg.distill_steps, desc='distilling (v11)')

    optimizer.zero_grad(set_to_none=True)

    while step < cfg.distill_steps:
        for batch in train_loader:
            if step >= cfg.distill_steps:
                break

            ids = batch[0].to(device, non_blocking=True)

            # =========================================================================
            # v11 TECHNIQUE 3: Get progressive stage parameters
            # =========================================================================
            if cfg.use_progressive_stages:
                stage_params = get_stage_params(step, cfg.distill_steps)
                stage = stage_params['stage']
                temp_target = stage_params['temp_target']
                align_mult = stage_params['align_mult']
            else:
                stage = 1
                temp_target = cfg.temperature
                align_mult = 1.0
            
            # Track stage transitions
            if stage != current_stage:
                print(f"\n  [stage transition] step {step}: stage {current_stage} -> stage {stage}")
                current_stage = stage

            with torch.cuda.amp.autocast():
                # =========================================================================
                # Teacher forward (with hidden states if alignment enabled)
                # =========================================================================
                with torch.no_grad():
                    if cfg.hidden_align_weight > 0:
                        t_out = teacher(ids, output_hidden_states=True)
                        t_logits = t_out.logits
                        t_hiddens = t_out.hidden_states
                    else:
                        t_logits = teacher(ids).logits

                # =========================================================================
                # Student forward (with hidden states if alignment enabled)
                # =========================================================================
                student.train()
                if cfg.hidden_align_weight > 0:
                    s_logits, s_hiddens = student(ids, return_hiddens=True)
                else:
                    s_logits = student(ids)

                # =========================================================================
                # v11 TECHNIQUE 1: Get temperature (learnable or fixed)
                # =========================================================================
                if temp_module is not None:
                    T = temp_module()
                else:
                    T = cfg.temperature

                # =========================================================================
                # KL divergence loss with temperature
                # =========================================================================
                s_log = F.log_softmax(s_logits / T, dim=-1)
                t_prob = F.softmax(t_logits / T, dim=-1)
                kl_loss = F.kl_div(
                    s_log.view(-1, s_logits.size(-1)),
                    t_prob.view(-1, t_logits.size(-1)),
                    reduction='batchmean'
                ) * (T ** 2)

                # =========================================================================
                # v11 TECHNIQUE 2: Hidden-state alignment (modulated by stage)
                # =========================================================================
                effective_align_weight = cfg.hidden_align_weight * align_mult
                
                if effective_align_weight > 0:
                    align_loss = compute_hidden_alignment_loss(
                        t_hiddens, s_hiddens, projector,
                        teacher_layers=cfg.teacher_n_layers,
                        student_layers=cfg.n_layers
                    )
                else:
                    align_loss = torch.tensor(0.0, device=device)

                # =========================================================================
                # v11: Temperature regularization (soft constraint towards stage target)
                # =========================================================================
                if temp_module is not None and cfg.use_progressive_stages:
                    temp_reg_loss = 0.1 * (T - temp_target) ** 2
                else:
                    temp_reg_loss = torch.tensor(0.0, device=device)

                # =========================================================================
                # v11 TECHNIQUE 4: Channel amplitude regularization
                # =========================================================================
                if cfg.use_channel_wise_spikes:
                    channel_reg = torch.tensor(0.0, device=device)
                    for layer in student.layers:
                        rec = layer['rec']
                        if hasattr(rec.k_spike, 'amplitude') and rec.k_spike.amplitude.numel() > 1:
                            channel_reg = channel_reg + rec.k_spike.amplitude.var()
                            channel_reg = channel_reg + rec.v_spike.amplitude.var()
                    channel_reg = channel_reg * 0.01
                else:
                    channel_reg = torch.tensor(0.0, device=device)

                # =========================================================================
                # Total loss with all components
                # =========================================================================
                loss = kl_loss + effective_align_weight * align_loss + temp_reg_loss + channel_reg

                # Scale loss for gradient accumulation
                loss = loss / accumulation_steps

            scaler.scale(loss).backward()
            accum_step += 1

            # Only step optimizer after accumulation_steps
            if accum_step % accumulation_steps == 0:
                scaler.unscale_(optimizer)
                gn = torch.nn.utils.clip_grad_norm_(all_params, cfg.max_grad_norm)

                if torch.isfinite(gn):
                    scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad(set_to_none=True)

                hw_stats.record_step(ids.size(0) * accumulation_steps, ids.size(1))
                spike_stats.record(student, step)

                current_lr = optimizer.param_groups[0]['lr']
                current_temp = temp_module.get_temperature() if temp_module is not None else cfg.temperature

                # Track losses (multiply back for logging)
                training_logs['loss_history'].append({'step': step, 'loss': loss.item() * accumulation_steps})
                training_logs['kl_loss_history'].append({'step': step, 'loss': kl_loss.item()})
                training_logs['align_loss_history'].append({'step': step, 'loss': align_loss.item() if isinstance(align_loss, torch.Tensor) else align_loss})
                training_logs['lr_history'].append({'step': step, 'lr': current_lr})
                
                # v11: Track new metrics
                training_logs['temp_history'].append({'step': step, 'temperature': current_temp})
                training_logs['stage_history'].append({'step': step, 'stage': stage})
                training_logs['channel_reg_history'].append({'step': step, 'loss': channel_reg.item() if isinstance(channel_reg, torch.Tensor) else channel_reg})

                pbar.set_postfix(
                    loss=f"{loss.item() * accumulation_steps:.3f}",
                    kl=f"{kl_loss.item():.3f}",
                    T=f"{current_temp:.2f}",
                    stg=stage,
                    lr=f"{current_lr:.1e}"
                )
                pbar.update(1)
                step += 1

                if step % cfg.eval_interval == 0:
                    val_loss = evaluate(student, val_loader, device)
                    val_ppl = get_ppl(val_loss)
                    training_logs['ppl_history'].append({'step': step, 'ppl': val_ppl})

                    amps = student.get_amplitudes()
                    amp_str = ', '.join([f"L{i}:{amps[f'layer_{i}']['k']:.2f}" for i in range(min(4, cfg.n_layers))])
                    align_str = f", align={align_loss.item():.4f}" if effective_align_weight > 0 else ""
                    print(f"\n  step {step}: ppl={val_ppl:.1f}, T={current_temp:.2f}, stage={stage}, amps=[{amp_str}...]{align_str}")

                    if val_loss < best_val:
                        best_val = val_loss
                        save_dict = {
                            'student': student.state_dict(),
                            'projector': projector.state_dict(),
                        }
                        if temp_module is not None:
                            save_dict['temp_module'] = temp_module.state_dict()
                        torch.save(save_dict, f'{OUTPUT_DIR}/checkpoints/v11_best.pt')

    pbar.close()
    return training_logs

print("distillation function defined (v11 - ALL FOUR ADVANCED TECHNIQUES)")


In [ ]:
# =============================================================================
# cell 18: run distillation (v11 - ALL FOUR TECHNIQUES)
# =============================================================================
print("="*60)
print("phase 1: distillation (v11 - ALL FOUR ADVANCED TECHNIQUES)")
print("="*60)
print("")
print("v11 architecture (balanced for speed):")
print(f"  d_model: 512 → {config.d_model} (-12%)")
print(f"  n_layers: 8 → {config.n_layers} (-25%)")
print(f"  params: ~51M → ~{student_params // 1_000_000}M (-27%)")
print("")
print("v11 ADVANCED TECHNIQUES:")
print(f"  1. learnable temperature (CTKD): {config.use_learnable_temperature}")
print(f"  2. hidden alignment: weight={config.hidden_align_weight} (v7 used 1.0, FAILED)")
print(f"  3. progressive stages (POCL): {config.use_progressive_stages}")
print(f"  4. channel-wise spikes: {config.use_channel_wise_spikes}")
print("")
print("speedups (same as v10):")
print(f"  gradient checkpointing: {USE_GRADIENT_CHECKPOINTING}")
print(f"  torch.compile: {compile_success}")
print(f"  accumulation_steps: {config.accumulation_steps}")
print(f"  fused optimizer: enabled")
print("")

hw_stats = HardwareStatsCollector()
spike_stats = SpikeStatsCollector(config.n_layers)

distill_logs = distill_v11(
    teacher, student, projector, train_loader, val_loader,
    config, DEVICE, hw_stats, spike_stats
)

print("")
print(f"distillation complete!")
print(f"throughput: {hw_stats.get_summary()['throughput_tokens_per_sec']:.0f} tokens/sec")
print("")

# v11: Show final temperature and stage
if config.use_learnable_temperature and distill_logs['temp_history']:
    final_temp = distill_logs['temp_history'][-1]['temperature']
    print(f"final temperature: {final_temp:.3f} (started at {config.temperature})")

if config.use_progressive_stages and distill_logs['stage_history']:
    final_stage = distill_logs['stage_history'][-1]['stage']
    print(f"final stage: {final_stage}")

print("")
print("final amplitudes (first 4 layers):")
for i, (k, v) in enumerate(student.get_amplitudes().items()):
    if i < 4:
        print(f"  {k}: k={v['k']:.4f}, v={v['v']:.4f}")
print(f"  ... ({config.n_layers - 4} more layers)")


## 5. test-time training (ttt) with lora

### 5.1 motivation

test-time training adapts the model to new data distributions at inference time. lora (hu et al., 2022) provides efficient adaptation.

### 5.2 note on current ttt

current ttt trains on the same distribution (wikitext-2 val). future versions should implement "triggered ttt" on out-of-distribution data per paper section 7.2.

In [ ]:
# =============================================================================
# cell 20: lora implementation (same as v9)
# =============================================================================
class LoRALinear(nn.Module):
    """lora adapter for linear layers."""

    def __init__(self, in_features, out_features, rank=8, alpha=16.0):
        super().__init__()
        self.scaling = alpha / rank
        self.lora_A = nn.Parameter(torch.zeros(rank, in_features))
        self.lora_B = nn.Parameter(torch.zeros(out_features, rank))
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))

    def forward(self, x):
        return (x @ self.lora_A.T @ self.lora_B.T) * self.scaling


def apply_lora(model, rank=8, alpha=16.0, targets=['key_proj', 'value_proj']):
    """apply lora adapters to specified modules."""
    lora_modules = {}
    for name, module in model.named_modules():
        if any(t in name for t in targets) and isinstance(module, nn.Linear):
            lora = LoRALinear(module.in_features, module.out_features, rank, alpha).to(next(module.parameters()).device)
            lora_modules[name] = lora
            orig_forward = module.forward
            def make_forward(orig, lora_mod):
                def forward(x):
                    return orig(x) + lora_mod(x)
                return forward
            module.forward = make_forward(orig_forward, lora)
    print(f"lora: {len(lora_modules)} modules, rank={rank}")
    return lora_modules

print("lora defined")

In [ ]:
# =============================================================================
# cell 21: ttt with lora (same as v9)
# =============================================================================
print("="*60)
print("phase 2: test-time training with lora")
print("="*60)

for p in student.parameters():
    p.requires_grad = False

lora_modules = apply_lora(student, config.lora_rank, config.lora_alpha)
lora_params = sum(p.numel() for m in lora_modules.values() for p in m.parameters())

pre_ttt_loss = evaluate(student, val_loader, DEVICE)
pre_ttt_ppl = get_ppl(pre_ttt_loss)
print(f"\npre-ttt ppl: {pre_ttt_ppl:.2f}")

lora_opt = torch.optim.AdamW([p for m in lora_modules.values() for p in m.parameters()], lr=config.ttt_lr)
ttt_logs = {'loss_history': []}
student.train()

for step, batch in enumerate(val_loader):
    if step >= config.ttt_steps:
        break
    ids = batch[0].to(DEVICE)
    with torch.cuda.amp.autocast():
        loss = F.cross_entropy(student(ids)[:, :-1].reshape(-1, config.vocab_size), ids[:, 1:].reshape(-1))
    lora_opt.zero_grad()
    loss.backward()
    lora_opt.step()
    ttt_logs['loss_history'].append({'step': step, 'loss': loss.item()})
    if step % 20 == 0:
        print(f"  ttt {step}: loss={loss.item():.4f}")

post_ttt_loss = evaluate(student, val_loader, DEVICE)
post_ttt_ppl = get_ppl(post_ttt_loss)
print(f"\npost-ttt ppl: {post_ttt_ppl:.2f}")
print(f"ttt improvement: {pre_ttt_ppl - post_ttt_ppl:.1f} ppl")

In [ ]:
# =============================================================================
# cell 22: final evaluation
# =============================================================================
print("="*60)
print("final evaluation")
print("="*60)

teacher_loss = evaluate(teacher, val_loader, DEVICE, is_gpt2=True)
teacher_ppl = get_ppl(teacher_loss)
student_loss = evaluate(student, val_loader, DEVICE)
student_ppl = get_ppl(student_loss)

# v10: VRAM logging
vram_peak_gb = torch.cuda.max_memory_allocated() / 1e9 if torch.cuda.is_available() else 0

print(f"")
print(f"{'model':<25} {'ppl':>10} {'params':>15}")
print("-" * 50)
print(f"{'gpt-2 (teacher)':<25} {teacher_ppl:>10.2f} {teacher_params:>15,}")
print(f"{'asnn-goose v10 (student)':<25} {student_ppl:>10.2f} {student_params:>15,}")
print("-" * 50)
print(f"{'compression':<25} {compression_ratio:>10.1f}x")
print(f"{'ppl gap':<25} {student_ppl - teacher_ppl:>10.2f}")
print(f"{'spike density':<25} {student.get_avg_spike_density():>10.3f}")
print(f"{'VRAM peak':<25} {vram_peak_gb:>10.2f}GB")
print("")
print("version comparison:")
print(f"  v6: 627.3 PPL (baseline)")
print(f"  v7: 1655 PPL (regression!)")
print(f"  v8: 559 PPL (fixed)")
print(f"  v9: 541.7 PPL (capacity increase)")
print(f"  v10: {student_ppl:.2f} PPL (100M model)")
if student_ppl < 480:
    print(f"  v10 TARGET MET! PPL < 480")
elif student_ppl < 541.7:
    print(f"  v10 beats v9 by {541.7 - student_ppl:.1f} PPL")
else:
    print(f"  WARNING: v10 did not improve over v9")

In [ ]:
# =============================================================================
# cell 23: visualization
# =============================================================================
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# distillation loss
d_steps = [l['step'] for l in distill_logs['loss_history']]
d_losses = [l['loss'] for l in distill_logs['loss_history']]
kl_losses = [l['loss'] for l in distill_logs['kl_loss_history']]
axes[0,0].plot(d_steps, d_losses, label='total', alpha=0.8)
axes[0,0].plot(d_steps, kl_losses, label='kl', alpha=0.7)
axes[0,0].set_xlabel('step')
axes[0,0].set_ylabel('loss')
axes[0,0].set_title('distillation loss (v10)')
axes[0,0].legend()

# validation ppl
p_steps = [l['step'] for l in distill_logs['ppl_history']]
p_ppls = [l['ppl'] for l in distill_logs['ppl_history']]
axes[0,1].plot(p_steps, p_ppls, 'orange', marker='o')
axes[0,1].axhline(y=teacher_ppl, color='green', linestyle='--', label=f'teacher ({teacher_ppl:.1f})')
axes[0,1].axhline(y=627.3, color='blue', linestyle=':', label='v6 (627.3)')
axes[0,1].axhline(y=541.7, color='purple', linestyle=':', label='v9 (541.7)')
axes[0,1].axhline(y=480, color='red', linestyle='--', label='v10 target (480)')
axes[0,1].set_xlabel('step')
axes[0,1].set_ylabel('ppl')
axes[0,1].set_title('validation ppl')
axes[0,1].legend()

# lr schedule
lr_steps = [l['step'] for l in distill_logs['lr_history']]
lr_vals = [l['lr'] for l in distill_logs['lr_history']]
axes[0,2].plot(lr_steps, lr_vals, 'purple')
axes[0,2].axvline(x=config.warmup_steps, color='gray', linestyle='--', label=f'warmup ({config.warmup_steps})')
axes[0,2].set_xlabel('step')
axes[0,2].set_ylabel('lr')
axes[0,2].set_title('learning rate')
axes[0,2].legend()

# spike density + amplitudes (first 4 layers)
spike_summary = spike_stats.get_summary()
layers = [f'layer_{i}' for i in range(min(4, config.n_layers))]
k_dens = [spike_summary['per_layer'][l]['k_final'] for l in layers]
v_dens = [spike_summary['per_layer'][l]['v_final'] for l in layers]
k_amps = [spike_summary['per_layer'][l]['k_amp_final'] for l in layers]
v_amps = [spike_summary['per_layer'][l]['v_amp_final'] for l in layers]

x = np.arange(len(layers))
axes[1,0].bar(x - 0.2, k_dens, 0.4, label='k density')
axes[1,0].bar(x + 0.2, v_dens, 0.4, label='v density')
ax2 = axes[1,0].twinx()
ax2.plot(x, k_amps, 'r-o', label='k amp')
ax2.plot(x, v_amps, 'b-s', label='v amp')
axes[1,0].set_xlabel('layer')
axes[1,0].set_ylabel('density')
ax2.set_ylabel('amplitude')
axes[1,0].set_title(f'spike density & amps (first 4/{config.n_layers} layers)')
axes[1,0].legend(loc='upper left')
ax2.legend(loc='upper right')

# ttt loss
t_steps = [l['step'] for l in ttt_logs['loss_history']]
t_losses = [l['loss'] for l in ttt_logs['loss_history']]
axes[1,1].plot(t_steps, t_losses, 'red')
axes[1,1].set_xlabel('step')
axes[1,1].set_ylabel('ce loss')
axes[1,1].set_title('ttt with lora')

# version comparison
versions = ['v6', 'v7', 'v8', 'v9', 'v10']
t_ppls = [44.6, 44.6, 44.6, 44.6, teacher_ppl]
s_ppls = [627.3, 1655, 559, 541.7, student_ppl]
x = np.arange(len(versions))
axes[1,2].bar(x - 0.2, t_ppls, 0.4, label='teacher', alpha=0.7)
axes[1,2].bar(x + 0.2, s_ppls, 0.4, label='student', alpha=0.7)
axes[1,2].axhline(y=480, color='red', linestyle='--', label='v10 target', alpha=0.7)
axes[1,2].set_xticks(x)
axes[1,2].set_xticklabels(versions)
axes[1,2].set_ylabel('ppl')
axes[1,2].set_title('version comparison')
axes[1,2].legend()
axes[1,2].set_yscale('log')

plt.tight_layout()
figure_path = f'{OUTPUT_DIR}/figures/v10_training_{RUN_TIMESTAMP}.png'
plt.savefig(figure_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"saved: {figure_path}")

In [ ]:
# =============================================================================
# cell 24: build results dict (DRAFT - validation_tests added in cell 26)
# =============================================================================
print("building results (draft - validation_tests added after cell 26)...")

with open(figure_path, 'rb') as f:
    figure_base64 = base64.b64encode(f.read()).decode('utf-8')

results = {
    'version': 'v10',
    'timestamp': datetime.now().isoformat(),
    'run_id': RUN_TIMESTAMP,
    'platform': PLATFORM,
    'description': '100M model with speedups (512d, 8L)',

    'v10_design': {
        'principle': '100M model with training speedups',
        'changes': {
            'd_model': '320 → 512 (+60%)',
            'n_layers': '5 → 8 (+60%)',
            'params': '~22M → ~100M (+350%)',
            'teacher_indices': '[2,5,7,10,12] → [1,2,4,5,7,8,10,11]',
        },
        'speedups': {
            'gradient_checkpointing': USE_GRADIENT_CHECKPOINTING,
            'torch_compile': compile_success,
            'fused_optimizer': True,
            'accumulation_steps': config.accumulation_steps,
        },
        'unchanged': [
            'temperature: 2.0',
            'hidden_align_weight: 0.0',
            'warmup_steps: 50',
            'distill_steps: 3000',
        ],
    },

    'architecture': {
        'teacher': {'name': 'gpt2', 'params': teacher_params},
        'student': {
            'name': 'asnn-goose-v10',
            'd_model': config.d_model,
            'n_layers': config.n_layers,
            'params': student_params,
        },
        'projector_params': projector_params,
        'compression_ratio': compression_ratio,
        'vram_peak_gb': vram_peak_gb,
    },

    'training_config': {
        'distill_steps': config.distill_steps,
        'temperature': config.temperature,
        'hidden_align_weight': config.hidden_align_weight,
        'warmup_steps': config.warmup_steps,
        'batch_size': config.batch_size,
        'accumulation_steps': config.accumulation_steps,
        'effective_batch': config.batch_size * config.accumulation_steps,
        'distill_lr': config.distill_lr,
        'max_grad_norm': config.max_grad_norm,
    },

    'results': {
        'teacher_ppl': teacher_ppl,
        'student_ppl': student_ppl,
        'ppl_gap': student_ppl - teacher_ppl,
        'spike_density': student.get_avg_spike_density(),
        'amplitudes': student.get_amplitudes(),
        'target_met': student_ppl < 480,
    },

    'training_curves': {
        'loss_history': distill_logs['loss_history'],
        'kl_loss_history': distill_logs['kl_loss_history'],
        'align_loss_history': distill_logs['align_loss_history'],
        'ppl_history': distill_logs['ppl_history'],
        'lr_history': distill_logs['lr_history'],
    },

    'hardware_stats': hw_stats.get_summary(),
    'spike_analysis': spike_stats.get_summary(),

    'ttt': {
        'lora_params': lora_params,
        'pre_ppl': pre_ttt_ppl,
        'post_ppl': post_ttt_ppl,
        'improvement': pre_ttt_ppl - post_ttt_ppl,
        'loss_history': ttt_logs['loss_history'],
    },

    'comparison': {
        'v6': {'student_ppl': 627.3, 'note': 'baseline'},
        'v7': {'student_ppl': 1655, 'note': 'regression (align=1.0, T=4)'},
        'v8': {'student_ppl': 559, 'note': 'fixed defaults (align=0, T=2)'},
        'v9': {'student_ppl': 541.7, 'note': 'capacity increase (320d, 5L)'},
        'v10': {'student_ppl': student_ppl, 'note': '100M model (512d, 8L)'},
    },

    'figures': {
        'training_plot': {
            'filename': f'v10_training_{RUN_TIMESTAMP}.png',
            'base64': figure_base64,
        }
    },
    
    # validation_tests will be added in cell 26
}

print("results dict built (validation_tests pending)")

In [ ]:
# =============================================================================
# cell 25: validation tests (v11 - 12 total: 9 existing + 3 new)
# =============================================================================
print("="*60)
print("validation tests (v11 - 12 tests)")
print("="*60)

test_results = {}

# 1. teacher pre-trained
print("\n[1] teacher pre-trained")
test_results['teacher_pretrained'] = teacher_ppl < 50
print(f"  teacher ppl: {teacher_ppl:.2f} - {'pass' if test_results['teacher_pretrained'] else 'fail'}")

# 2. student learned (v11 target: <510)
print("\n[2] student learned language")
test_results['student_learned'] = student_ppl < 627
print(f"  student ppl: {student_ppl:.2f} - {'pass' if test_results['student_learned'] else 'fail'} (target: < 627)")

# 3. ternary activations
print("\n[3] ternary activations")
student.eval()
with torch.no_grad():
    test_ids = next(iter(val_loader))[0].to(DEVICE)
    layer = student.layers[0]['rec']
    x = student.embed(test_ids) + student.pos_embed(torch.arange(test_ids.size(1), device=DEVICE).unsqueeze(0))
    x_norm = layer.ln(x)
    prev_x = F.pad(x_norm[:, :-1, :], (0, 0, 1, 0))
    xk = x_norm * layer.time_mix_k + prev_x * (1 - layer.time_mix_k)
    k_spike = layer.k_spike(layer.key_proj(xk))
    unique_vals = len(set([round(v, 4) for v in k_spike.unique().cpu().tolist()]))
    test_results['ternary'] = unique_vals <= 3
    print(f"  unique values: {unique_vals} - {'pass' if test_results['ternary'] else 'fail'}")

# 4. gradient flow
print("\n[4] gradient flow (STE)")
_spike = TrainableTernarySpike().to(DEVICE)
_x = torch.randn(2, 16, 64, device=DEVICE, requires_grad=True)
_spike(_x).sum().backward()
test_results['gradient'] = _x.grad is not None and _x.grad.abs().sum() > 0
print(f"  {'pass' if test_results['gradient'] else 'fail'}")

# 5. spike density
print("\n[5] spike density in range")
density = student.get_avg_spike_density()
test_results['density'] = 0.1 < density < 0.9
print(f"  density: {density:.3f} - {'pass' if test_results['density'] else 'fail'}")

# 6. lora applied
print("\n[6] lora applied")
test_results['lora'] = len(lora_modules) > 0
print(f"  modules: {len(lora_modules)} - {'pass' if test_results['lora'] else 'fail'}")

# 7. improvement over v6 baseline
print("\n[7] beats v6 baseline")
test_results['beats_v6'] = student_ppl < 627.3
print(f"  v6: 627.3, v11: {student_ppl:.2f} - {'pass' if test_results['beats_v6'] else 'fail'}")

# 8. amplitudes learned
print("\n[8] amplitudes learned")
amps = student.get_amplitudes()
test_results['amplitudes_learned'] = any(
    abs(amps[f'layer_{i}']['k'] - 1.0) > 0.05 or abs(amps[f'layer_{i}']['v'] - 1.0) > 0.05
    for i in range(config.n_layers)
)
amp_strs = [f"L{i}:{amps[f'layer_{i}']['k']:.3f}" for i in range(config.n_layers)]
print(f"  amplitudes: {amp_strs}")
print(f"  {'pass' if test_results['amplitudes_learned'] else 'fail'} - any amplitude != 1.0 by > 0.05")

# 9. amplitude health check
print("\n[9] amplitude health")
all_healthy = True
for layer_idx, amp_dict in amps.items():
    k_amp, v_amp = amp_dict['k'], amp_dict['v']
    if not (0.3 < k_amp < 3.0) or not (0.3 < v_amp < 3.0):
        print(f"  WARNING: {layer_idx} unhealthy: k={k_amp:.3f}, v={v_amp:.3f}")
        all_healthy = False
test_results['amplitude_health'] = all_healthy
print(f"  {'pass' if all_healthy else 'fail'} - all amplitudes in [0.3, 3.0]")

# =============================================================================
# v11 NEW TESTS (10, 11, 12)
# =============================================================================

# 10. Temperature evolution (CTKD)
print("\n[10] temperature evolution (v11 - CTKD)")
if config.use_learnable_temperature and 'temp_history' in distill_logs:
    temps = [h['temperature'] for h in distill_logs['temp_history']]
    if len(temps) > 100:
        # Temperature should change (not stuck at initial value)
        temp_range = max(temps) - min(temps)
        temp_changed = temp_range > 0.1
        
        # Final temperature should be in valid range [1.0, 10.0]
        final_temp = temps[-1]
        temp_in_range = 1.0 <= final_temp <= 10.0
        
        # Temperature should stabilize (low variance in last 20%)
        late_temps = temps[int(len(temps) * 0.8):]
        temp_stable = np.std(late_temps) < 1.0  # Allow some variance
        
        test_results['temperature_evolution'] = temp_changed and temp_in_range and temp_stable
        print(f"  temp range: {min(temps):.3f} - {max(temps):.3f} (changed: {temp_changed})")
        print(f"  final temp: {final_temp:.3f} (in range: {temp_in_range})")
        print(f"  late variance: {np.std(late_temps):.4f} (stable: {temp_stable})")
        print(f"  {'pass' if test_results['temperature_evolution'] else 'fail'}")
    else:
        test_results['temperature_evolution'] = False
        print(f"  not enough data points ({len(temps)}) - fail")
else:
    test_results['temperature_evolution'] = True  # Pass if disabled (feature not used)
    print(f"  learnable temperature disabled - pass (skipped)")

# 11. Alignment loss tracking
print("\n[11] alignment loss tracking (v11)")
if config.hidden_align_weight > 0 and 'align_loss_history' in distill_logs:
    align_losses = [h['loss'] for h in distill_logs['align_loss_history']]
    if len(align_losses) > 100:
        # Filter out zeros (from stage 1 where alignment is disabled)
        nonzero_losses = [l for l in align_losses if l > 0]
        
        if len(nonzero_losses) > 50:
            # Alignment loss should decrease over time (when active)
            first_quarter = np.mean(nonzero_losses[:len(nonzero_losses)//4])
            last_quarter = np.mean(nonzero_losses[-len(nonzero_losses)//4:])
            align_decreased = last_quarter < first_quarter
            
            test_results['alignment_loss_tracking'] = align_decreased
            print(f"  first quarter mean: {first_quarter:.4f}")
            print(f"  last quarter mean: {last_quarter:.4f}")
            print(f"  decreased: {align_decreased}")
            print(f"  {'pass' if test_results['alignment_loss_tracking'] else 'fail'}")
        else:
            test_results['alignment_loss_tracking'] = True  # Pass if not enough non-zero points
            print(f"  not enough non-zero points ({len(nonzero_losses)}) - pass (skipped)")
    else:
        test_results['alignment_loss_tracking'] = False
        print(f"  not enough data points ({len(align_losses)}) - fail")
else:
    test_results['alignment_loss_tracking'] = True  # Pass if disabled
    print(f"  hidden alignment disabled - pass (skipped)")

# 12. Stage transitions (POCL)
print("\n[12] stage transitions (v11 - POCL)")
if config.use_progressive_stages and 'stage_history' in distill_logs:
    stages = [h['stage'] for h in distill_logs['stage_history']]
    if len(stages) > 100:
        # Should have all 3 stages
        has_all_stages = 1 in stages and 2 in stages and 3 in stages
        
        # Stage transitions at approximately correct points
        if has_all_stages:
            try:
                stage1_end = next(i for i, s in enumerate(stages) if s == 2)
                stage2_end = next(i for i, s in enumerate(stages) if s == 3)
                total = len(stages)
                
                timing_ok = (0.35 < stage1_end/total < 0.45) and (0.65 < stage2_end/total < 0.75)
                
                test_results['stage_transitions'] = has_all_stages and timing_ok
                print(f"  stage 1 ended at: {stage1_end}/{total} ({100*stage1_end/total:.1f}%)")
                print(f"  stage 2 ended at: {stage2_end}/{total} ({100*stage2_end/total:.1f}%)")
                print(f"  timing correct: {timing_ok}")
                print(f"  {'pass' if test_results['stage_transitions'] else 'fail'}")
            except StopIteration:
                test_results['stage_transitions'] = False
                print(f"  could not find stage transitions - fail")
        else:
            test_results['stage_transitions'] = False
            unique_stages = set(stages)
            print(f"  missing stages (found: {unique_stages}) - fail")
    else:
        test_results['stage_transitions'] = False
        print(f"  not enough data points ({len(stages)}) - fail")
else:
    test_results['stage_transitions'] = True  # Pass if disabled
    print(f"  progressive stages disabled - pass (skipped)")

print("\n" + "="*60)
passed = sum(1 for v in test_results.values() if v)
print(f"results: {passed}/{len(test_results)} passed")
if student_ppl < 510:
    print(f"v11 TARGET MET: PPL {student_ppl:.2f} < 510")
elif student_ppl < 526.4:
    print(f"v11 improved from v10: {526.4 - student_ppl:.1f} PPL reduction")


In [ ]:
# =============================================================================
# cell 26: FINAL save with validation_tests (v10 bug fix)
# =============================================================================
print("="*60)
print("FINAL SAVE (includes validation_tests)")
print("="*60)

# Add validation_tests to results
results['validation_tests'] = test_results

# Save final results.json with ALL data
results_path = f'{OUTPUT_DIR}/results/results_{RUN_TIMESTAMP}.json'
with open(results_path, 'w') as f:
    json.dump(results, f, indent=2, default=str)

print(f"saved: {results_path}")
print(f"size: {os.path.getsize(results_path) / 1024:.1f} KB")
print(f"")
print(f"validation_tests included: {list(test_results.keys())}")

# v10: auto-download AFTER final save
print("")
print("auto-download")
if IS_COLAB:
    try:
        from google.colab import files
        files.download(results_path)
        files.download(figure_path)
        print("downloads started!")
    except Exception as e:
        print(f"download failed: {e}")
elif IS_KAGGLE:
    print(f"kaggle: {results_path}")
else:
    print(f"local: {results_path}")

## 6. summary

### 6.1 v11 design principle

**ALL FOUR ADVANCED TECHNIQUES (37M params)**

| attribute | v10 | v11 | change |
|-----------|-----|-----|--------|
| d_model | 512 | **448** | -12% |
| n_layers | 8 | **6** | -25% |
| params | ~100M | **~37M** | -63% |
| temperature | fixed 2.0 | **learnable (CTKD)** | adaptive |
| hidden_align_weight | 0.0 | **0.01** | enabled |
| progressive stages | none | **3 stages (POCL)** | curriculum |
| ternary spikes | global | **channel-wise** | per-channel |

### 6.2 four advanced techniques

| technique | paper | benefit |
|-----------|-------|---------|
| **Learnable Temperature (CTKD)** | ArXiv 2211.16231 | adaptive difficulty |
| **Light Hidden Alignment** | TinyBERT 1909.10351 | intermediate supervision |
| **Progressive Stages (POCL)** | ArXiv 2506.05695 | curriculum learning |
| **Channel-wise Ternary** | TerViT/TTQ | per-channel expressivity |

### 6.3 version progression

| version | teacher ppl | student ppl | key change |
|---------|-------------|-------------|------------|
| v6 | 44.6 | **627** | gpt-2 distillation |
| v7 | 44.6 | 1655 | **regression** (align=1.0, T=4) |
| v8 | 44.6 | **559** | fixed defaults |
| v9 | 44.6 | **541.7** | capacity increase (320d, 5L) |
| v10 | ~45 | **526.4** | 100M model + speedups |
| **v11** | ~45 | **target: <510** | **ALL FOUR TECHNIQUES** |

### 6.4 validation tests (12 total)

**original tests (1-9):**
1. teacher pre-trained (PPL < 50)
2. student learned (PPL < 627)
3. ternary activations verified
4. gradient flow via STE
5. spike density in [0.1, 0.9]
6. LoRA applied
7. beats v6 baseline
8. amplitudes learned
9. amplitude health [0.3, 3.0]

**v11 new tests (10-12):**
10. temperature evolution (CTKD) - T changes during training
11. alignment loss tracking - decreases over time
12. stage transitions (POCL) - correct timing at 40%, 70%

### 6.5 success criteria

| metric | target |
|--------|--------|
| PPL | **<510** |
| tests | **12/12 pass** |
| temperature | evolves (not stuck at 2.0) |
| alignment loss | decreases over training |
| stage transitions | at correct steps (40%, 70%) |
| channel amplitudes | in [0.3, 3.0] range |

---

*asnn-goose v11 - eptesicus laboratories - lumis-next initiative*

### references

- hinton, g., vinyals, o., & dean, j. (2015). distilling the knowledge in a neural network.
- radford, a., et al. (2019). language models are unsupervised multitask learners.
- jiao et al. (2020). tinybert: distilling bert for natural language understanding.
- li et al. (2022). curriculum temperature for knowledge distillation (CTKD).
- progressive overload curriculum learning (POCL, 2025).
- trained ternary quantization (TTQ, 2017).